## Document Classification

For this assignment I used text messages that were labeled as 'spam' or 'ham' (https://raw.githubusercontent.com/wtznc/Naive-Bayes-SMS-Spam-Collection/master/SMSSpamCollection).
This problem belong to supervised learning.  The goal of this assignment is to find the classifier that can predict whether text mesage belong to 'spam' or 'ham' category with higher accuracy.  

In [142]:
#import packages
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier


I read data from scv file.

In [ ]:
df = pd.read_csv('spam.csv',names=['category','text message'],skiprows=1)
df.head()

In order to use tf-idf model on the text I did the following:

1. Converted all upper case letters to lowerr case letters.
2. Removed punctuations.
3. Removed English stopwordds such as articles and prepositions.

In [ ]:
# remove punctuation
df['text message'] = df['text message'].str.replace(r'[^\w\s]','')
df.head()

In [140]:
# remove stopwords
stop_words = set(stopwords.words('english'))

df['tokenized_text'] = df['text message'].apply(word_tokenize) 
for i in range(0,len(df['tokenized_text'])):
    df['tokenized_text'][i] =[w for w in df['tokenized_text'][i] if w not in stop_words]
    df['tokenized_text'][i] = ' '.join(w for w in df['tokenized_text'][i])
    
df.head()

,category,text message,tokenized_text
0,ham,Go until jurong point crazy Available only in ...,Go jurong point crazy Available bugis n great ...
1,ham,Ok lar Joking wif u oni,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry 2 wkly comp win FA Cup final tkts 2...
3,ham,U dun say so early hor U c already then say,U dun say early hor U c already say
4,ham,Nah I dont think he goes to usf he lives aroun...,Nah I dont think goes usf lives around though


Split data into two datasets - training and testing

In [132]:
X = df.iloc[:,2].values
y = df.iloc[:,0].values
train_X, test_X, train_y, test_y  = train_test_split(X, y, test_size=0.3)
train_X, val_X, train_y, val_y  = train_test_split(X, y, test_size=0.3)

I applied Random Forest classifier.

In [ ]:
regressor = RandomForestClassifier(n_estimators =80,class_weight ='balanced')
regressor.fit(train_X,train_y)
pred = regressor.predict(test_X)
F1score[name]= f1_score(test_y, train_y)
Acc[name] = accuracy_score(test_y,pred)
confusion_mat[name] = confusion_matrix(test_y,pred)
predictions[name]=pred
print(name+': Accuracy=%1.3f, F1=%1.3f'%(Acc[name],F1score[name]))